This code is to test how many hidden layers and hidden nodes could achieve
the nonlinear spring damper property of torque controller.

Feb. 07, 2018  
Huawei Wang

In [1]:
import plotly
plotly.tools.set_credentials_file(username='HuaweiWang', api_key='u3JPHaMgbggGVFml1yUd')

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

In [3]:
num_nodes = 5001
duration = 100.0
num_states = 6
num_tors = 3

In [4]:
Inputs = 1 - 2*np.random.random((num_states, num_nodes))
Outputs = np.zeros((num_tors, num_nodes))
time = np.linspace(0, duration, num_nodes)

Con = np.zeros((num_tors, num_states))

In [5]:
#K0 = np.zeros((num_tors, num_states))

#K0[:, :3] = 500 + 300*np.random.random((num_tors, 3))
#K0[:, 3:] = 100 + 80*np.random.random((num_tors, 3))

In [6]:
K0 = np.zeros((num_tors, num_states))

K0[:, :3] = 100*np.eye(num_tors)
K0[:, 3:] = 10*np.eye(num_tors)
K0

array([[ 100.,    0.,    0.,   10.,    0.,    0.],
       [   0.,  100.,    0.,    0.,   10.,    0.],
       [   0.,    0.,  100.,    0.,    0.,   10.]])

In [7]:
Ks = np.array([5000, 4400, 3800])

for k in range(num_nodes):
    Con[0, [0, 3]] = K0[0, [0, 3]] + Ks[0]*abs(Inputs[[0, 3], k])**3
    Con[1, [1, 4]] = K0[1, [1, 4]] + Ks[1]*abs(Inputs[[1, 4], k])**3
    Con[2, [2, 5]] = K0[2, [2, 5]] + Ks[2]*abs(Inputs[[2, 5], k])**3
           
    Outputs[:, k] = np.dot(Con, Inputs[:, k])

In [8]:
np.random.seed()
Test_inp = 1 - 2*np.random.random((num_states, num_nodes))
Test_out = np.zeros((num_tors, num_nodes))

for m in range(num_nodes):
    
    Con[0, [0, 3]] = K0[0, [0, 3]] + Ks[0]*abs(Test_inp[[0, 3], m])**3
    Con[1, [1, 4]] = K0[1, [1, 4]] + Ks[1]*abs(Test_inp[[1, 4], m])**3
    Con[2, [2, 5]] = K0[2, [2, 5]] + Ks[2]*abs(Test_inp[[2, 5], m])**3
    
    Test_out[:, m] = np.dot(Con, Test_inp[:, m])

In [9]:
trace1 = go.Scatter3d(
    x = Inputs[0, :],
    y = Inputs[3, :],
    z = Outputs[0,:],
    mode = 'markers',
    marker=dict(
        size=2,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x = Inputs[1, :],
    y = Inputs[4, :],
    z = Outputs[1,:],
    mode = 'markers',
    marker=dict(
    color='rgb(127, 127, 127)',
    size=2,
    symbol='circle',
    line=dict(
        color='rgb(204, 204, 204)',
        width=0.5
    ),
    opacity=0.9
    )
)

trace3 = go.Scatter3d(
    x = Inputs[2, :],
    y = Inputs[5, :],
    z = Outputs[2,:],
    mode = 'markers',
    marker=dict(
    color='rgb(187, 187, 187)',
    size=2,
    symbol='circle',
    opacity=0.9
    )
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)

In [10]:
py.iplot(fig, filename='torques')

In [11]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential

Using TensorFlow backend.


In [12]:
from keras.layers import Dense, Activation

In [13]:
# Initialize the constructor
model = Sequential()

In [14]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def custom_activation(x):
    if x < -1:
        out = -1
    elif x <= 1:
        out = x**3
    else:
        out = 1
    return out

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [15]:
# Add an input layer 
model.add(Dense(8, activation=custom_activation, input_shape=(6,)))

# Add Output layer 
model.add(Dense(3, activation=custom_activation))

#model.add(Activation(custom_activation))

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [14]:
# Add an input layer 
model.add(Dense(8, activation='sigmoid', input_shape=(6,)))

# Add Output layer 
model.add(Dense(3, activation='sigmoid'))

In [16]:
model.compile(loss = 'mean_squared_error', optimizer = 'sgd')

In [17]:
for j in range(1):
    model.fit(Inputs.T, Outputs.T/(abs(Outputs).max().max()), epochs = 10, batch_size = 100)

Epoch 1/10
5001/5001 [==============================] - 0s 34us/step - loss: nan
Epoch 2/10
5001/5001 [==============================] - 0s 20us/step - loss: nan
Epoch 3/10
5001/5001 [==============================] - 0s 21us/step - loss: nan
Epoch 4/10
5001/5001 [==============================] - 0s 24us/step - loss: nan
Epoch 5/10
5001/5001 [==============================] - 0s 23us/step - loss: nan
Epoch 6/10
5001/5001 [==============================] - 0s 23us/step - loss: nan
Epoch 7/10
5001/5001 [==============================] - 0s 22us/step - loss: nan
Epoch 8/10
5001/5001 [==============================] - 0s 19us/step - loss: nan
Epoch 9/10
5001/5001 [==============================] - 0s 23us/step - loss: nan
Epoch 10/10
5001/5001 [==============================] - 0s 21us/step - loss: nan


In [28]:
score = model.evaluate(Test_inp.T, Test_out.T/(abs(Outputs).max().max()), batch_size = 500)

5001/5001 [==============================] - 0s 15us/step


In [29]:
score

0.050178120373469831

In [38]:
(abs(Inputs).max().max())

0.99999962075857751

In [30]:
predictions = model.predict(Test_inp.T)*(abs(Outputs).max().max())

In [31]:
trace7 = go.Scatter(
    x = Test_inp[0, :],
#    y = Test_inp[3, :],
    y = Test_out[0,:],
    mode = 'markers'
)

trace8 = go.Scatter(
    x = Test_inp[0, :],
#    y = Test_inp[3, :],
    y = predictions.T[0,:],
    mode = 'markers'
)


data2 = [trace7, trace8]
fig2 = go.Figure(data=data2, layout=layout)


In [32]:
Test_out[0,:]

array([ -116.16561404, -5318.73927273,   360.25826963, ...,  4773.5195159 ,
        1160.74302127,  -298.96186708])

In [35]:
predictions.T[0,:]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [33]:
py.iplot(fig2, filename='fit')

In [34]:
trace4 = go.Scatter3d(
    x = Test_inp[0, :],
    y = Test_inp[3, :],
    z = predictions.T[0,:],
    mode = 'markers',
    marker=dict(
        size=2,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.1
        ),
        opacity=0.8
    )
)

trace5 = go.Scatter3d(
    x = Test_inp[1, :],
    y = Test_inp[4, :],
    z = predictions.T[1,:],
    mode = 'markers',
    marker=dict(
    color='rgb(127, 127, 127)',
    size=2,
    symbol='circle',
    line=dict(
        color='rgb(204, 204, 204)',
        width=0.5
    ),
    opacity=0.9
    )
)

trace6 = go.Scatter3d(
    x = Test_inp[2, :],
    y = Test_inp[5, :],
    z = predictions.T[2,:],
    mode = 'markers',
    marker=dict(
    color='rgb(187, 187, 0)',
    size=2,
    symbol='circle',
    opacity=0.9
    )
)

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

data1 = [trace4, trace5, trace6]
fig1 = go.Figure(data=data1, layout=layout)

In [109]:
py.iplot(fig1, filename='fit')